In [9]:
import torch
import random
import copy
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import optimizer
from torchvision import datasets, transforms
from matplotlib import pyplot as plt


In [10]:
MNIST = datasets.MNIST(root=".",
                       train=True,
                       transform = transforms.ToTensor(),
                       download=False)

MNIST_VAL = datasets.MNIST(root=".",
                           train=False,
                           transform = transforms.ToTensor(),
                           download=False)                    

In [11]:
dado, rotulo = MNIST[0]
print(dado.size())
print(MNIST.data.shape)

torch.Size([1, 28, 28])
torch.Size([60000, 28, 28])


In [12]:
dataloader = DataLoader(MNIST, batch_size=64)
dataloader_val = DataLoader(MNIST_VAL, batch_size=64)

In [13]:
class MLPLeakyReLU(nn.Module):
    def __init__(self):
        super(MLPLeakyReLU, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(784, 10),
            nn.LeakyReLU(),
            nn.Linear(10, 10),
            nn.LeakyReLU(),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.layers(x)  

#Instanciando a Rede Neural
model = MLPLeakyReLU()

In [15]:
def achatar(X):
    batch_size = X.size(0)
    flattened_X = X.view(batch_size, -1)
    return flattened_X

def permutar(images):
    permuted_images = images.clone()
    for i in range(images.size(0)):
        image = images[i]
        permuted_indices = torch.randperm(image.numel())
        permuted_image = image.view(-1)[permuted_indices].view(image.size())
        permuted_images[i] = permuted_image
    return permuted_images

In [17]:
learningRate = 0.001
optimizer = torch.optim.Adam(model.parameters(),lr=learningRate)
loss_fn = nn.CrossEntropyLoss()
epochs = 10

for e in range(epochs):
    #LOOP DE TREINAMENTO
    for i, (X, y) in enumerate(dataloader):
        X = permutar(X)
        X = achatar(X)
        yhat = model(X)
        loss = loss_fn(yhat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossvalue, pos = loss.item(), i * len(X)
        if i % 1000 == 0:
            print(f"Epoch({e}:{pos}): [Loss: {lossvalue:.2f}]")
    #LOOP DE TESTE
    tloss = 0.0
    correct = 0.0
    size = len(dataloader_val.dataset)
    with torch.no_grad():
        for X, y in dataloader_val:
            X = achatar(X)
            pred = model(X)
            tloss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        print(f"Test error: \n Accuracy {100 * (correct/size):2.2f}%")
        

Epoch(0:0): [Loss: 2.23]
Test error: 
 Accuracy 20.93%
Epoch(1:0): [Loss: 2.21]
Test error: 
 Accuracy 20.17%
Epoch(2:0): [Loss: 2.21]
Test error: 
 Accuracy 21.12%
Epoch(3:0): [Loss: 2.20]
Test error: 
 Accuracy 21.25%
Epoch(4:0): [Loss: 2.21]
Test error: 
 Accuracy 22.47%
Epoch(5:0): [Loss: 2.20]
Test error: 
 Accuracy 21.75%
Epoch(6:0): [Loss: 2.19]
Test error: 
 Accuracy 22.18%
Epoch(7:0): [Loss: 2.19]
Test error: 
 Accuracy 21.90%
Epoch(8:0): [Loss: 2.19]
Test error: 
 Accuracy 21.56%
Epoch(9:0): [Loss: 2.19]
Test error: 
 Accuracy 21.41%
